# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

## Starbuck's Capstone Challenge
Instructions for the project can be found in the Starbucks Project Workspace.

### Dataset overview
- The program used to create the data simulates how people make purchasing decisions and how those decisions are influenced by promotional offers.
- Each person in the simulation has some hidden traits that influence their purchasing patterns and are associated with their observable traits. People produce various events, including receiving offers, opening offers, and making purchases.
- As a simplification, there are no explicit products to track. Only the amounts of each transaction or offer are recorded.
- There are three types of offers that can be sent: buy-one-get-one (BOGO), discount, and informational. In a BOGO offer, a user needs to spend a certain amount to get a reward equal to that threshold amount. In a discount, a user gains a reward equal to a fraction of the amount spent. In an informational offer, there is no reward, but neither is there a requisite amount that the user is expected to spend. Offers can be delivered via multiple channels.
- <span style="color:blue">The basic task is to use the data to identify which groups of people are most responsive to each type of offer, and how best to present each type of offer.

### Data Dictionary

#### profile.json
Rewards program users (17000 users x 5 fields)

gender: (categorical) M, F, O, or null
age: (numeric) missing value encoded as 118
id: (string/hash)
became_member_on: (date) format YYYYMMDD
income: (numeric)
#### portfolio.json
Offers sent during 30-day test period (10 offers x 6 fields)

reward: (numeric) money awarded for the amount spent
channels: (list) web, email, mobile, social
difficulty: (numeric) money required to be spent to receive reward
duration: (numeric) time for offer to be open, in days
offer_type: (string) bogo, discount, informational
id: (string/hash)
#### transcript.json
Event log (306648 events x 4 fields)

person: (string/hash)
event: (string) offer received, offer viewed, transaction, offer completed
value: (dictionary) different values depending on event type
offer id: (string/hash) not associated with any "transaction"
amount: (numeric) money spent in "transaction"
reward: (numeric) money gained from "offer completed"
time: (numeric) hours after start of test

In [2]:
# import libraries
import pandas as pd
import json

In [3]:
# create a Spark session

with open('mini_sparkify_event_data_comma.json') as f:
    json_data = json.load(f)
df = pd.DataFrame(json_data)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [4]:
for col in df.columns:
    print(col, df[col].unique())
# df['method'].unique()

artist ['Martha Tilston' 'Five Iron Frenzy' 'Adam Lambert' ..., 'PROFANE OMEN'
 'Gabriel Romero' '28 Days']
auth ['Logged In' 'Logged Out' 'Cancelled' 'Guest']
firstName ['Colin' 'Micah' 'Ashlynn' 'Alexi' nan 'Warren' 'Alex' 'Ainsley' 'Zyonna'
 'Faigy' 'Madison' 'Grant' 'Anika' 'Isobella' 'Payton' 'Christian' 'Teagan'
 'Kael' 'Daelin' 'Giovanni' 'Jaleel' 'Panav' 'Carly' 'Elizabeth' 'Riley'
 'Deegan' 'Ethan' 'Molly' 'Lucero' 'Bryson' 'Nicole' 'Kaylenn' 'Joseph'
 'Oliver' 'Annabella' 'Ellen' 'Madelyn' 'Sawyer' 'Chase' 'Spencer'
 'Sebastian' 'Dillyn' 'Lina' 'Calvin' 'Erick' 'Autumn' 'Miracle' 'Kaleb'
 'Chayse' 'Parker' 'Adelaida' 'Lily' 'Sadie' 'Aurora' 'Freedom' 'Genesis'
 'William' 'Davis' 'Nikolas' 'Peyton' 'Nithya' 'Adriel' 'Brayden' 'Jayden'
 'Alexander' 'Alexandria' 'Destiny' 'Nathan' 'Jeffery' 'Sophia' 'Brisa'
 'Rachel' 'Kayla' 'Jaxon' 'Maddox' 'Diego' 'Titus' 'Jada' 'Emma' 'Christan'
 'Isabella' 'Garrett' 'Piper' 'Justin' 'Amberlynn' 'Zoey' 'Caleb' 'Makayla'
 'Logan' 'Lorelei' 'Al

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [10]:
print(df.shape)
df = df.dropna(subset=['userId','sessionId'],thresh=1)
print(df.shape)

(286500, 18)
(286500, 18)


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.